In [1]:
from flask import Flask, request, redirect
import secrets
import os
import Spotify_Client



Authentication: I prove who I say I am -> username & password

Authorization: I prove that someone else told me I could acess something -> Oauth


In [2]:
CLIENT_ID = os.environ['Spotify_Client_ID']
CLIENT_SECRET = os.environ['Spotify_Client_Secret']

Spotify = Spotify_Client.Spotify_API(CLIENT_ID, CLIENT_SECRET)

secret_key = secrets.token_urlsafe(16)

app = Flask(__name__)
app.secret_key = secret_key


In [3]:
@app.route("/")
def index():
    # Authorization
    auth_url = Spotify.User_Oauth()
    return redirect(auth_url)

@app.route("/data")
def callback():
    print('callback')
    Spotify.Access_Refresh_token()

    #Gathering of profile data
    print('profile_data')
    profile_data = Spotify.Profile_Data()
    print(profile_data)
    
    return profile_data

In [4]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [04/Apr/2021 16:50:55] "GET / HTTP/1.1" 302 -


callback
profile_data


127.0.0.1 - - [04/Apr/2021 16:50:57] "GET /data?code=AQASFs4lz0kfd0MFJcoMEqBk4WvLZQBfWj5tAT8ajy0_7nnfGB38vy9Z7dj1IZAykBRi3DKmk45rb9lXSvrz-Y617-G8n3ohTG3cuDqGZNgMxPQESfRKZrHPhEWsNCBEdwYZy-V2kMEnTwt9U5FU8Drt7UzKZEzBnYWe7M0UUzI_J_js7eTgM9t-NTpWTP0ROd1vMyAC_A7MJG_B7BjGFN_MD__-KjLQLi_Tn-6IORm4s08MeqL6a10LZn0K77Vng7PhS0MVUrjCpuI HTTP/1.1" 200 -


{'display_name': 'helloagain0', 'external_urls': {'spotify': 'https://open.spotify.com/user/helloagain0'}, 'followers': {'href': None, 'total': 1}, 'href': 'https://api.spotify.com/v1/users/helloagain0', 'id': 'helloagain0', 'images': [], 'type': 'user', 'uri': 'spotify:user:helloagain0'}


In [ ]:
#Spotify.search(query = 'Hangman', search_type = 'track')
#Spotify.search({'track' : 'Hangman', 'artist': "Dave"}, search_type = 'track')
#Spotify.search(query= 'Hangman', operator ='not', operator_query='Tom', search_type = 'track')
#Spotify.get_User_Top_Data('artists','long_term')

BUILDING Graph

In [ ]:
'''
domain = ['hip hop','grime', 'edm', 'pop', 'country', 'jazz','rock','classical','r&b','metal','rap']
dominant = set(domain)
difgenres = set()
user_top = Spotify.get_User_Top_Data('artists','medium_term')
for items in user_top['items']:
    print(items['name'])
    print(items['genres'])
    flag = 0
    temp = None
    genres = items['genres']
    for genre in genres:
        if genre in dominant and flag != 1:
            print(genre)
            difgenres.add(genre)
            flag = 1
            break
        else:
            if flag == 0:
                temp = genre
                flag = -1
                
    if flag == -1:
        if temp == "pop dance":
            temp = 'edm'
        elif temp == 'sad rap':
            temp = 'emo rap'

        elif temp == 'indie hip hop':
            temp = 'hip hop'
        difgenres.add(temp)
            
print(difgenres)
'''

In [5]:
Spotify.get_artist('6Ip8FS7vWT1uKkJSweANQK')

{'external_urls': {'spotify': 'https://open.spotify.com/artist/6Ip8FS7vWT1uKkJSweANQK'},
 'followers': {'href': None, 'total': 1261435},
 'genres': ['uk hip hop'],
 'href': 'https://api.spotify.com/v1/artists/6Ip8FS7vWT1uKkJSweANQK',
 'id': '6Ip8FS7vWT1uKkJSweANQK',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/ab6761610000e5eb5dfa06c05a11cd16e15f0836',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/ab676161000051745dfa06c05a11cd16e15f0836',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/ab6761610000f1785dfa06c05a11cd16e15f0836',
   'width': 160}],
 'name': 'Dave',
 'popularity': 77,
 'type': 'artist',
 'uri': 'spotify:artist:6Ip8FS7vWT1uKkJSweANQK'}

In [5]:
import graph_builder
test = graph_builder.build_graph()
net = graph_builder.visualize_graph(test)


Performing Authorization
Building Relations for:  Kanye West
Building Relations for:  Drake
Building Relations for:  Stormzy
Building Relations for:  Flume
Building Relations for:  Martin Garrix
Building Relations for:  Dave
Building Relations for:  Jon Bellion
Building Relations for:  G-Eazy
Building Relations for:  BROCKHAMPTON
Building Relations for:  Aries
Building Relations for:  Quinn XCII
Building Relations for:  Sylvan LaCue
Building Relations for:  blackbear
Building Relations for:  J. Cole
Building Relations for:  Avicii
Building Relations for:  Big Sean
Building Relations for:  Post Malone
Building Relations for:  Lil Baby
Building Relations for:  AJ Tracey
Building Relations for:  Nipsey Hussle
Building Relations for:  Bugzy Malone
Building Relations for:  JAY-Z
Building Relations for:  YSN Fab
Building Relations for:  Zedd
Building Relations for:  The Manor
Building Relations for:  Dominic Fike
Building Relations for:  Meek Mill
Building Relations for:  Jeremy Zucker
Build

Adding Related Artist:  Sylvan LaCue
Adding Related Artist:  Sylvan LaCue
Adding Related Artist:  Sylvan LaCue
Adding Related Artist:  Sylvan LaCue
Adding Related Artist:  Sylvan LaCue
Adding Related Artist:  Sylvan LaCue
Adding Related Artist:  Sylvan LaCue
Adding Related Artist:  Sylvan LaCue
Adding Related Artist:  Sylvan LaCue
Adding Related Artist:  Sylvan LaCue
Adding Related Artist:  blackbear
Adding Related Artist:  blackbear
Adding Related Artist:  blackbear
Adding Related Artist:  blackbear
Adding Related Artist:  blackbear
Adding Related Artist:  blackbear
Adding Related Artist:  blackbear
Adding Related Artist:  blackbear
Adding Related Artist:  J. Cole
Adding Related Artist:  J. Cole
Adding Related Artist:  J. Cole
Adding Related Artist:  J. Cole
Adding Related Artist:  J. Cole
Adding Related Artist:  Avicii
Adding Related Artist:  Avicii
Adding Related Artist:  Avicii
Adding Related Artist:  Avicii
Adding Related Artist:  Avicii
Adding Related Artist:  Avicii
Adding Relat

Adding Related Artist:  XXXTENTACION
Adding Related Artist:  XXXTENTACION
Adding Related Artist:  XXXTENTACION
Adding Related Artist:  XXXTENTACION
Adding Related Artist:  XXXTENTACION
Adding Related Artist:  XXXTENTACION
Adding Related Artist:  Dizzee Rascal
Adding Related Artist:  Dizzee Rascal
Adding Related Artist:  Dizzee Rascal
Adding Related Artist:  Dizzee Rascal
Adding Related Artist:  Dizzee Rascal
Adding Related Artist:  Dizzee Rascal
Adding Related Artist:  Dizzee Rascal
Adding Related Artist:  Lil Mosey
Adding Related Artist:  Lil Mosey
Adding Related Artist:  Lil Mosey
Adding Related Artist:  Lil Mosey
Adding Top Artist:  Creedence Clearwater Revival
Adding Related Artist:  Creedence Clearwater Revival
Adding Related Artist:  Creedence Clearwater Revival
Adding Related Artist:  Creedence Clearwater Revival
Adding Related Artist:  Creedence Clearwater Revival
Adding Related Artist:  Creedence Clearwater Revival
Adding Related Artist:  Creedence Clearwater Revival
Adding Re